In [12]:
import time
start_time = time.time()

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pandas import Series
from sklearn.ensemble import RandomForestRegressor

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [14]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [15]:
train.isnull().sum().sum()

In [16]:
test.isnull().sum().sum()

In [17]:
train.columns

In [18]:
test.columns

In [19]:
#taken from https://bigdatascientistblog.wordpress.com/2015/10/02/feature-engineering-with-dates-part-1/
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
day = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).day
hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
minute = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).minute

seasons = [0,0,1,1,1,2]
season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)]

In [20]:
#from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

#dr = pd.date_range(start='2016-01-01', end='2016-06-30')
#df = pd.DataFrame()
#df['Date'] = dr

#cal = calendar()
#holidays = cal.holidays(start=dr.min(), end=dr.max())

#df['Holiday'] = df['Date'].isin(holidays)

In [21]:
#for train data
train['month'] = train['pickup_datetime'].map(month)
train['day'] = train['pickup_datetime'].map(day)
train['day_of_week'] = train['pickup_datetime'].map(day_of_week)
train['hour'] = train['pickup_datetime'].map(hour)
train['minute'] = train['pickup_datetime'].map(minute)
train['season'] = train['pickup_datetime'].map(season)

#train['Date2'] = pd.to_datetime(train['pickup_datetime'], errors='coerce')
#train['holiday'] = train['Date2'].dt.date.astype('datetime64[ns]').isin(holidays)
#train=train.drop(['Date2'], axis=1)

train['store_and_fwd_flag']=train['store_and_fwd_flag'].map( {'N': 0, 'Y': 1} ).astype(int)

In [22]:
#for test data
test['month'] = test['pickup_datetime'].map(month)
test['day_of_week'] = test['pickup_datetime'].map(day_of_week)
test['day'] = test['pickup_datetime'].map(day)
test['hour'] = test['pickup_datetime'].map(hour)
test['minute'] = test['pickup_datetime'].map(minute)
test['season'] = test['pickup_datetime'].map(season)

#test['Date2'] = pd.to_datetime(test['pickup_datetime'], errors='coerce')
#test['holiday'] = test['Date2'].dt.date.astype('datetime64[ns]').isin(holidays)
#test = test.drop(['Date2'], axis=1)

test['store_and_fwd_flag']=test['store_and_fwd_flag'].map( {'N': 0, 'Y': 1} ).astype(int)

In [23]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
Series(train['month']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Month")
plt.ylabel("Count")
plt.title('Which month has the most rides? (train data)')
plt.subplot(1, 2, 2)
Series(test['month']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Month")
plt.ylabel("Count")
plt.title('Which month has the most rides? (test data)')

In [24]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
Series(train['day']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Day")
plt.ylabel("Count")
plt.title('Which day has the most rides?(train data)')
plt.subplot(1, 2, 2)
Series(test['day']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Day")
plt.ylabel("Count")
plt.title('Which day has the most rides?(test data)')

In [25]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
Series(train['day_of_week']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Day of week")
plt.ylabel("Count")
plt.title('Which day of week has the most rides? \n (0 = Monday)-(train data)')
plt.subplot(1, 2, 2)
Series(test['day_of_week']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Day of week")
plt.ylabel("Count")
plt.title('Which day of week has the most rides? \n (0 = Monday)-(test data)')

In [26]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
Series(train['hour']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Hour")
plt.ylabel("Count")
plt.title('What hour has the most taxi rides? (train data)')
plt.subplot(1, 2, 2)
Series(test['hour']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Hour")
plt.ylabel("Count")
plt.title('What hour has the most taxi rides? (test data)')

In [27]:
plt.figure(figsize=(10,10))
plt.subplot(2, 1, 1)
Series(train['minute']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Minute")
plt.ylabel("Count")
plt.title('What minute has the most taxi rides order? (train data)')
plt.subplot(2, 1, 2)
Series(test['minute']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Minute")
plt.ylabel("Count")
plt.title('What minute has the most taxi rides order? (test data)')

In [28]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
Series(train['season']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Season")
plt.ylabel("Count")
plt.title('Which season has the most taxi rides order? \n (0 = Winter Season, 1=Spring Season ,2= Summer Season) \n (train data)')
plt.subplot(1, 2, 2)
Series(test['season']).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel("Season")
plt.ylabel("Count")
plt.title('Which season has the most taxi rides order? \n (0 = Winter Season, 1=Spring Season ,2= Summer Season) \n (test data)')

In [29]:
#plt.figure(figsize=(12,5))
#plt.subplot(1, 2, 1)
#Series(train['holiday']).value_counts().sort_index().plot(kind = 'bar')
#plt.ylabel("Count")
#plt.title('How many holydays/non-holidays \n has the most taxi rides?(train data)')
#plt.subplot(1, 2, 2)
#Series(test['holiday']).value_counts().sort_index().plot(kind = 'bar')
#plt.ylabel("Count")
#plt.title('How many holydays/non-holidays \n has the most taxi rides?(test data)')

In [30]:
from geopy.distance import great_circle
def distance(p_lat,p_long,d_lat,d_long):
    pickup = (p_lat, p_long)
    dropoff = (d_lat, d_long)
    distance_all=great_circle(pickup, dropoff).km
    return distance_all

In [31]:
#for train data
d_train = [distance(train['pickup_latitude'].values[i], train['pickup_longitude'].values[i],
              train['dropoff_latitude'].values[i],train['dropoff_longitude'].values[i]) for i in range(len(train['pickup_latitude']))]
train['distance_est']=d_train

In [32]:
train["d1"]=train['distance_est']<=5
train["d2"]=train['distance_est'].between(5, 10, inclusive=False)
train["d3"]=train['distance_est'].between(10, 20, inclusive=False)
train["d4"]=train['distance_est'].between(20, 30, inclusive=False)
train["d5"]=train['distance_est'].between(30, 100, inclusive=False)
train["d6"]=train['distance_est']>100

In [33]:
#for test data
d_test = [distance(test['pickup_latitude'].values[i], test['pickup_longitude'].values[i],
              test['dropoff_latitude'].values[i],test['dropoff_longitude'].values[i]) for i in range(len(test['pickup_latitude']))]
test['distance_est']=d_test

In [34]:
test["d1"]=test['distance_est']<=5
test["d2"]=test['distance_est'].between(5, 10, inclusive=False)
test["d3"]=test['distance_est'].between(10, 20, inclusive=False)
test["d4"]=test['distance_est'].between(20, 30, inclusive=False)
test["d5"]=test['distance_est'].between(30, 100, inclusive=False)
test["d6"]=test['distance_est']>100

In [35]:
#dimension reduction of pickup and dropoff (not sure about this step)
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=1, random_state=1)

In [36]:
#for train data
frames_p_train=[train['pickup_latitude'],train['pickup_longitude']]
p_train=pd.concat(frames_p_train,axis=1)

frames_d_train=[train['dropoff_latitude'],train['dropoff_longitude']]
d_train=pd.concat(frames_d_train,axis=1)

train['tsvd_p']=tsvd.fit_transform(p_train)
train['tsvd_d']=tsvd.fit_transform(d_train)

In [37]:
#for test data
frames_p_test=[test['pickup_latitude'],test['pickup_longitude']]
p_test=pd.concat(frames_p_test,axis=1)

frames_d_test=[test['dropoff_latitude'],test['dropoff_longitude']]
d_test=pd.concat(frames_d_test,axis=1)

test['tsvd_p']=tsvd.fit_transform(p_test)
test['tsvd_d']=tsvd.fit_transform(d_test)

In [38]:
#direction
# taken from: https://gist.github.com/jeromer/2005586
import math

def calculate_initial_compass_bearing(pointA, pointB):
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [39]:
#for train data
train['direction']=[calculate_initial_compass_bearing((train['pickup_latitude'].loc[i],train['pickup_longitude'].loc[i]), 
                                  (train['dropoff_latitude'].loc[i],train['dropoff_longitude'].loc[i])) for i in range(len(train['pickup_latitude']))]

In [40]:
#for test data
test['direction']=[calculate_initial_compass_bearing((test['pickup_latitude'].loc[i],test['pickup_longitude'].loc[i]), 
                                  (test['dropoff_latitude'].loc[i],test['dropoff_longitude'].loc[i])) for i in range(len(test['pickup_latitude']))]

In [41]:
sns.heatmap(train.corr())

In [42]:
test_id=test["id"]
test = test.drop(['id', 'pickup_datetime'], axis=1)
X = train.drop(['id','trip_duration', 'pickup_datetime','dropoff_datetime'], axis=1)
Y = train['trip_duration']

In [43]:
RF = RandomForestRegressor(verbose=True)
RF.fit(X, Y)

In [44]:
features_list = X.columns.values
feature_importance = RF.feature_importances_
sorted_idx = np.argsort(feature_importance)
 
plt.figure(figsize=(5,10))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), features_list[sorted_idx])
plt.xlabel('Importance')
plt.title('Feature importances')
plt.draw()
plt.show()

In [45]:
#work in progress for improved results (currently the LB result is bad)
Y_pred = RF.predict(test)

In [46]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['trip_duration'] = Y_pred
sub.to_csv('RF.csv', index=False)

In [47]:
print(" %s seconds " % (time.time() - start_time))